# Requirement:
- numpy 1.19.5
- tensorflow 2.5.0
- tensorflow_privacy 0.6.1
- sklearn 0.24.2

Before performing the experiments, you need to fit the **mia_path** (path for MIA package) and **pickle_file**(path for tabular QMNIST data) according to your own PC.

In [1]:
import numpy as np

import pickle
import tensorflow as tf
from tensorflow.keras import layers
import random as python_random

from sklearn.model_selection import train_test_split

np.random.seed(20)
python_random.seed(123)
tf.random.set_seed(3)

NUM_CLASSES = 10

# Load QMNIST data.

In [2]:
pickle_file = '/home/jiangnan/Desktop/dataset/QMNIST_tabular.pickle'

with open(pickle_file, 'rb') as f:
  pickle_data = pickle.load(f)
  x_defender = pickle_data['x_private']
  x_reserve = pickle_data['x_reserved']
  y_defender = pickle_data['y_private']
  y_reserve = pickle_data['y_reserved']
  del pickle_data
print('Data loaded.')

Data loaded.


In [3]:
NUM_CLASSES = 10

y_defender = y_defender[:,0]
y_reserve = y_reserve[:,0]

y_defender = np.expand_dims(y_defender,axis=1)
y_reserve = np.expand_dims(y_reserve,axis=1)

y_defender = tf.keras.utils.to_categorical(y_defender, num_classes=NUM_CLASSES)
y_reserve = tf.keras.utils.to_categorical(y_reserve, num_classes=NUM_CLASSES)

# Defender model $M_D$

In [4]:
#l2_norm_clip = 1.0
#noise_multiplier = 1.1

epochs = 5
batch_size = 64
validation_split = 0.5

def defender_model_fn():
    """The architecture of the defender (victim) model.
    The attack is white-box, hence the attacker is assumed to know this architecture too."""
    
    tf.random.set_seed(3)
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(NUM_CLASSES, activation=tf.nn.softmax)
    ])
    
    ##if train_with_DP:
    ##from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
    ##train_op = DPKerasSGDOptimizer(
    ##    l2_norm_clip=l2_norm_clip,
    ##    noise_multiplier=noise_multiplier,
    ##    num_microbatches=1, # Possible problem after reducing the size of cost vector in tensorflow-privacy. Check: https://github.com/tensorflow/privacy/issues/17
    ##    learning_rate=1e-4
    ##    )
    ##else:
    
    train_op = tf.optimizers.Adam(1e-4)
    
    model.compile(optimizer=train_op,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


## Train $M_D$ 

In [5]:
number_records = [48, 100, 200, 400, 800, 1600, 3200, 6400, 12800, 25600, 51200, 102400, 200000]
number_records_used = number_records[1]

data_in = x_defender[:number_records_used], y_defender[:number_records_used]
data_out = x_reserve[:number_records_used], y_reserve[:number_records_used]

In [6]:
defender_model_path = '/home/jiangnan/Desktop/ppml-workshop/defender_trained_models/simple_fn_100examples'
defender_model = tf.keras.models.load_model(defender_model_path)

## Evaluate the utility of $M_D$ on utility evaluation dataset $E_U$ (which is equal to reserve dataset $R$).

Evaluation metrics: **Accuracy** & **AUC**

In [7]:
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import euclidean_distances

In [8]:
# compute the accuracy as acc
predict_reserve_proba = defender_model.predict_proba(data_out[0])
predict_reserve = np.argmax(predict_reserve_proba, axis=1)
label_reserve = np.argmax(data_out[1], axis=1)
acc = accuracy_score(label_reserve, predict_reserve)

/home/jiangnan/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [9]:
M_cD_in = defender_model_fn()

M_cD_in.fit(data_in[0], data_in[1],epochs = 5,batch_size = 64,validation_split = 0.5,verbose = False)

M_cD_in_predict = M_cD_in.predict_proba(data_in[0])


/home/jiangnan/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [10]:
predict = defender_model.predict_proba(data_in[0])

In [11]:
predict[0]

array([8.2086276e-19, 9.9996698e-01, 1.6368784e-05, 3.0216087e-17,
       5.5897314e-19, 5.2853290e-20, 2.5350758e-10, 1.3587395e-05,
       3.3737153e-18, 3.0875349e-06], dtype=float32)

In [12]:
M_cD_in_predict[0]

array([8.2086276e-19, 9.9996698e-01, 1.6368784e-05, 3.0216087e-17,
       5.5897314e-19, 5.2853290e-20, 2.5350758e-10, 1.3587421e-05,
       3.3737153e-18, 3.0875349e-06], dtype=float32)

In [108]:
auc_by_class = []

# compute auc per class then take the average value
for i in range(NUM_CLASSES):
  class_indices = np.argmax(y_reserve, axis=1) == i
  fpr, tpr, thresholds = metrics.roc_curve(class_indices, predict_reserve_proba[:,i])
  auc = metrics.auc(fpr, tpr)
  auc_by_class.append(auc)

average_auc = np.mean(auc_by_class)

ValueError: Found input variables with inconsistent numbers of samples: [80600, 400]

In [109]:
print('Utility of defender model:')
print('Acc: {}'.format(acc))
print('Auc: {}'.format(average_auc))

Utility of defender model:
Acc: 0.1525
Auc: 0.45392531552025767


In [110]:
# if necessary, save or reload the defender model trained

#defender_model_path = '/home/jiangnan/Desktop/model/QMNIST_defender_model'
#defender_model.save(defender_model_path)

#defender_model = tf.keras.models.load_model(defender_model_path)

# Oracle attack model $M_A$

In [13]:
import random
from tqdm import tqdm

In [22]:
def create_mock_defender_models(n_samples = 0, given_index = 0):
    
    similarities_in = []
    similarities_out = []

    for i in tqdm(range(data_in[0].shape[0])):

        #index = i
        index = random.randint(0,number_records[0]-1)

        evaluation_data_in = data_in[0][index]
        evaluation_label_in = data_in[1][index]

        evaluation_data_out = data_out[0][index]
        evaluation_label_out = data_out[1][index]

        evaluation_data = np.array([evaluation_data_in, evaluation_data_out])
        evaluation_label = np.array([evaluation_label_in, evaluation_label_out])

        evaluation = evaluation_data, evaluation_label


        attack_train_data_in = np.delete(data_in[0], index, axis=0)
        attack_train_label_in = np.delete(data_in[1], index, axis=0)

        attack_in = attack_train_data_in, attack_train_label_in


        attack_train_data_out = np.delete(data_out[0], index, axis=0)
        attack_train_label_out = np.delete(data_out[1], index, axis=0)

        attack_out = attack_train_data_out, attack_train_label_out


        predict = defender_model.predict_proba(attack_in[0])

        if given_index == 1:

            attack_in_plus_one_in = np.insert(attack_in[0], index, evaluation[0][0].reshape(1,attack_in[0].shape[1]), axis=0), np.insert(attack_in[1], index, evaluation[1][0], axis=0)
            attack_in_plus_one_out = np.insert(attack_in[0], index, evaluation[0][1].reshape(1,attack_in[0].shape[1]), axis=0), np.insert(attack_in[1], index, evaluation[1][1], axis=0)

        else:

            attack_in_plus_one_in = np.vstack((evaluation[0][0].reshape(1,attack_in[0].shape[1]),attack_in[0])), np.vstack(( evaluation[1][0].reshape(1,attack_in[1].shape[1]),attack_in[1]))
            attack_in_plus_one_out = np.vstack((evaluation[0][1].reshape(1,attack_in[0].shape[1]),attack_in[0])), np.vstack(( evaluation[1][1].reshape(1,attack_in[1].shape[1]),attack_in[1]))

        M_cD_in = defender_model_fn()
        M_cD_out = defender_model_fn()

        M_cD_in.fit(attack_in_plus_one_in[0], attack_in_plus_one_in[1],epochs = 5,batch_size = 64,validation_split = 0.5,verbose = False)
        M_cD_out.fit(attack_in_plus_one_out[0], attack_in_plus_one_out[1],epochs = 5,batch_size = 64,validation_split = 0.5,verbose = False)

        M_cD_in_predict = M_cD_in.predict_proba(attack_in[0])
        M_cD_out_predict = M_cD_out.predict_proba(attack_in[0])

        similarity_in = np.mean(np.linalg.norm(M_cD_in_predict-predict, axis=1))
        similarity_out = np.mean(np.linalg.norm(M_cD_out_predict-predict, axis=1))

        similarities_in.append(similarity_in)
        similarities_out.append(similarity_out)
    
    return similarities_in, similarities_out


In [23]:
similarities_in, similarities_out = create_mock_defender_models(n_samples = 0, given_index = 1)

100%|██████████| 100/100 [02:58<00:00,  1.78s/it]


In [24]:
similarities_out[:10]

[1.1452866,
 1.1452866,
 1.1413746,
 1.1446074,
 1.1435964,
 1.1452866,
 1.1544728,
 1.1452866,
 1.1456271,
 1.1412885]

In [25]:
similarities_in[:10]

[3.1054682e-08,
 3.1054682e-08,
 3.1054682e-08,
 3.1054686e-08,
 3.1054686e-08,
 3.1054682e-08,
 3.1054686e-08,
 3.1054682e-08,
 3.1017056e-08,
 3.1054686e-08]

In [26]:
results = []

for i in tqdm(range(len(similarities_in))):
    for j in range(len(similarities_out)):
        
        if similarities_in[i] < similarities_out[j]:
            results.append(1)
        else:
            results.append(0)

100%|██████████| 100/100 [00:00<00:00, 15582.36it/s]


In [27]:
n = len(results)
p = 1-np.sum(results)/n

privacy = min(2*p,1)
variance = 2*p*(1-p)/n
sigma_error = 2*np.sqrt(p*(1-p)/n)

In [28]:
print('Percentage of wrong guess: {} %'.format(p*100))
print('Defender model privacy: {}'.format(privacy))
print('Privacy variance: {}'.format(variance))
print('Privacy error: {}'.format(sigma_error))

Percentage of wrong guess: 0.0 %
Defender model privacy: 0.0
Privacy variance: 0.0
Privacy error: 0.0
